In [32]:
from tkinter import *
import matplotlib.pyplot as plt
#bigger fonts
import matplotlib
matplotlib.rc('xtick', labelsize=15) 
matplotlib.rc('ytick', labelsize=15)
matplotlib.rcParams.update({'font.size': 20})

matplotlib.use("TkAgg")

import pandas as pd
import numpy as np
import scipy.stats as ss

# Reading properties

In [11]:
T = pd.read_csv('../TOST_data/average_properties.txt',
            sep='\t',index_col=0,na_values=-99,
            dtype={'WEIGHT_GRAMS': float,
                   'GEST_AGE_DAYS':float,
                   'DELTA_SEC_MEASURE':float,
                   'MED_SPO2':float,
                   'MED_PI':float,
                   'MED_PR':float})

In [12]:
# GESTATIONAL AGE BINNING
slots = [0.,38*7,39*7,40*7,41*7,500.]
sel = []
for i in range(len(slots)-1):
    sel.append((T['GEST_AGE_DAYS']>= slots[i])&(T['GEST_AGE_DAYS']< slots[i+1]))

In [36]:
# TIME SINCE BIRTH BINNING
slots_hour = [0.,10,20,30,40,100.]
DELTA_HOURS = np.array(T['DELTA_SEC_MEASURE'])/60./60.
sel_delta_t = []
for i in range(len(slots_hour)-1):
    sel_delta_t.append((DELTA_HOURS>= slots_hour[i])&(DELTA_HOURS< slots_hour[i+1]))

In [101]:
np.unique(T['DELIVERY'])

array(['Csection', 'vaginal', 'vaginal assisted'], dtype=object)

# Functions

In [60]:
def plot_hist_split(prop='MED_PR',select=((T['MED_PR']>10)& (T['PHN']==0) & (T['FOOT']==1)),running_sel=sel,slots=np.array(slots)/7.):
    fig = plt.figure(figsize=(10,8))
    for i in range(len(running_sel)):
        data = T[prop][running_sel[i] & select]  
        plt.hist(data,bins=20,histtype='step',color='C%i'%i,lw=3,label='[%.1f,%.1f] mean=%.1f'%(slots[i],slots[i+1],np.mean(data)))
        plt.axvline(np.mean(data),color='C%i'%i,ls='dashed',lw=3)
    plt.xlabel(prop)
    plt.ylabel('counts')
    plt.legend(fontsize='x-small')
    #plt.title('NO PHN - ONLY FOOT')
    return fig

def plot_graph_split(prop='MED_PR',select=((T['MED_PR']>10)& (T['PHN']==0) & (T['FOOT']==1)),running_sel=sel,slots=np.array(slots)/7.,running_prop='GESTATIONAL_AGE',txt_y=100,txt_x=38,conversion=1/7.):
    fig = plt.figure(figsize=(10,8))
    y = []
    x = []
    xerr = []
    yerr = []

    for i in range(len(running_sel)):
        tot_sel = running_sel[i] & select
        data = T[prop][tot_sel]  
        
        if i==0:
            data_first = data
        elif i ==len(running_sel)-1:
            data_last = data
        
        y.append(np.mean(data))
        
        
        x.append(np.mean(T[running_prop][tot_sel])*conversion)
        #xerr.append(np.std(T[running_prop][tot_sel]/7.)/np.sqrt(len(data)))
        
        
        yerr.append(np.std(data))#/np.sqrt(len(data)) )
        #yerr.append(ss.iqr(data) )

        
        #plt.hist(data,bins=20,histtype='step',color='C%i'%i,lw=3,label='[%.1f,%.1f] mean=%.1f'%(slots[i],slots[i+1],np.mean(data)))
        #plt.axvline(np.mean(data),color='C%i'%i,ls='dashed',lw=3)
        print(len(data))
        #print(len(T[running_prop][tot_sel]))
    x = np.array(x)
    y = np.array(y)
    #xerr = np.array(xerr)
    yerr = np.array(yerr)
    
    #98,140
    plt.text(txt_x,txt_y,'difference extremes = %.2f, p-value %.6f'%(np.mean(data_first)-np.mean(data_last),ss.ttest_ind(data_first,data_last)[1]))
    
    #plt.errorbar(x,y,xerr=xerr,yerr=yerr,color='k',lw=5)
    plt.errorbar(x,y,yerr=yerr,color='k',lw=5)
    plt.plot(x,y,".",markersize=30,color='k')

    plt.xlabel(running_prop)
    plt.ylabel(prop)
    #plt.legend(fontsize='x-small')
    #plt.title('NO PHN - ONLY FOOT')
    return fig



In [118]:
def Update_lbl():
    lbl_prop.configure(text=str(prop.get()))
    
def plot_histo():
    plot_hist_split(str(prop.get()),select=(T[str(prop.get())]>10)& (T['PHN']==0) & (T['WRIST']==1),running_sel=sel)
    plt.xlabel(str(prop.get()))
    plt.title('NO PHN - ONLY FOOT')
    plt.show()
    
    
def plot_graph():
    plot_graph_split(str(prop.get()),select=(T[str(prop.get())]>10)& (T['PHN']==0) & (T['WRIST']==1),running_prop='GEST_AGE_DAYS',running_sel=sel,txt_y=136)
    plt.xlabel('GESTATIONAL AGE WEEKS')
    plt.title('NO PHN - ONLY WRIST')
    plt.show()

    
def stat():
    new_wind = Tk()
    new_wind.title('Mean values in bins')
    str_stat = ''
    for i in range(len(slots_hour)-1):
        str_stat += 'MEAN VALUES %i < delta_t <  %i [hours]'%(slots_hour[i],slots_hour[i+1])
        str_stat += '\n##################################\n'
        str_stat +=  str(T[sel_delta_t[i]].mean())+'\n\n'
    lbl_stat = Label(new_wind,text=str_stat,font=("Arial Bold", 9))
    lbl_stat.grid(row=0,column=0)
    new_wind.mainloop()

def Update_data():
    pass
    
window = Tk()
window.title("Prakash project NHS")
window.geometry('950x600')
lbl = Label(window,text='Select property to be plotted:',font=("Arial Bold", 15))#.grid(column=0,row=0)
lbl.grid(row=0,column=0)
prop = StringVar()
prop.set('MED_PR')
lbl_prop = Label(window, text= str(prop.get()))
lbl_prop.grid(row=0,column=4)
r1 = Radiobutton(window, text="Pulse Rate", variable=prop, value='MED_PR',indicatoron=0,command=Update_lbl)
r1.grid(row=0,column=1)
r2 = Radiobutton(window, text="Oxigen Saturation", variable=prop, value='MED_SPO2',indicatoron=0,command=Update_lbl)
r2.grid(row=0,column=2)
r3 = Radiobutton(window, text="Perfusion index", variable=prop, value='MED_PI',indicatoron=0,command=Update_lbl)
r3.grid(row=0,column=3)

# --- SELECTION ---

###PHN
phn = IntVar()
phn.set(-99)
rs1 = Radiobutton(window, text="ONLY PHN", variable=phn, value=1,indicatoron=1)
rs1.grid(row=1,column=0)
rs2 = Radiobutton(window, text="EXCLUDE PHN", variable=phn, value=0,indicatoron=1)
rs2.grid(row=1,column=1)
rs3 = Radiobutton(window, text="ALL", variable=phn, value=-99,indicatoron=1)
rs3.grid(row=1,column=2)

#### WRIST/FOOT
wrist = IntVar()
wrist.set(-99)
rs4 = Radiobutton(window, text="WRIST", variable=wrist, value=1,indicatoron=1)
rs4.grid(row=2,column=0)
rs5 = Radiobutton(window, text="FOOT", variable=wrist, value=0,indicatoron=1)
rs5.grid(row=2,column=1)
rs6 = Radiobutton(window, text="ALL", variable=wrist, value=-99,indicatoron=1)
rs6.grid(row=2,column=2)

#### BOYS and GIRL
gender = StringVar()
gender.set('All')
rs7 = Radiobutton(window, text="MALE", variable=gender, value='Male',indicatoron=1)
rs7.grid(row=3,column=0)
rs8 = Radiobutton(window, text="FEMALE", variable=gender, value='Female',indicatoron=1)
rs8.grid(row=3,column=1)
rs9 = Radiobutton(window, text="ALL", variable=gender, value='All',indicatoron=1)
rs9.grid(row=3,column=2)

### DELIVERY
delivery = StringVar()
delivery.set('All')
rs10 = Radiobutton(window, text="Vaginal delivery", variable=gender, value='vaginal',indicatoron=1)
rs10.grid(row=4,column=0)
rs11 = Radiobutton(window, text="Vaginal assisted", variable=gender, value='vaginal assisted',indicatoron=1)
rs11.grid(row=4,column=1)
rs12 = Radiobutton(window, text="Csection", variable=gender, value='Csection',indicatoron=1)
rs12.grid(row=4,column=2)
rs13 = Radiobutton(window, text="All", variable=gender, value='All',indicatoron=1)
rs13.grid(row=4,column=3)

###BUTTON TO SUBMIT SELECTION
btn_submit_sel = Button(window,text='Submit selection',bg='violet',fg='white',command=Update_data)
btn_submit_sel.grid(column=4,row=4)

### NEED to update data with the current selection 


btn_hist = Button(window,text='Histogram in bins',bg='blue',fg='white',command=plot_histo)
btn_hist.grid(column=0,row=10)

btn_plot = Button(window,text='Plot in bins',bg='red',fg='white',command=plot_graph)
btn_plot.grid(column=0,row=11)

btn_stat = Button(window,text='stats',bg='green',fg='white',command=stat)
btn_stat.grid(column=0,row=12)



In [119]:
window.mainloop()
print(phn.get())

24
69
67
79
40
1
